### Data Attributes

Let's focus on data attributes first (non-callables).

As we saw before we can have class attributes - they live in the class dictionary:

In [1]:
class BankAccount:
    apr = 1.2

In [2]:
BankAccount.__dict__

mappingproxy({'__module__': '__main__',
              'apr': 1.2,
              '__dict__': <attribute '__dict__' of 'BankAccount' objects>,
              '__weakref__': <attribute '__weakref__' of 'BankAccount' objects>,
              '__doc__': None})

In [3]:
BankAccount.apr

1.2

Now when we create instances of that class:

In [4]:
acc_1 = BankAccount()
acc_2 = BankAccount()

The instance dictionaries are currently empty:

In [5]:
acc_1.__dict__, acc_2.__dict__

({}, {})

Yet, these instances do have an `apr` attribute:

In [6]:
acc_1.apr, acc_2.apr

(1.2, 1.2)

Where is that value coming from? The class that the objects were created from!

In fact, if we modify the class attribute:

In [7]:
BankAccount.apr = 2.5

We'll see this reflected in the instances as well:

In [8]:
acc_1.apr, acc_2.apr

(2.5, 2.5)

In [9]:
acc_1.apr = 5

In [10]:
acc_1.__dict__, acc_2.__dict__

({'apr': 5}, {})

In [11]:
acc_1.apr, acc_2.apr

(5, 2.5)

And if we add a class attribute to `BankAccount`:

In [12]:
BankAccount.account_type = 'Savings'

In [13]:
acc_1.account_type, acc_2.account_type

('Savings', 'Savings')

As you can see, modifying attributes in the **class** are reflected in the instances too - that's because Python does not find an `apr` attribute in the instance dictionary, so next it looks in the class that was used to create the instance.

Which raises the question, what happens if we add `apr` to the **instance** dictionary?

In [14]:
acc_1.apr = 0

Well that did not raise an exception - so what's happening now:

In [15]:
acc_1.__dict__, acc_2.__dict__

({'apr': 0}, {})

As you can see, we actually created an entry for `apr` in the state dictionary of `acc_1`.

Now that we have it there, it we try to get the attribute value `apr` for `acc_1`, Python will find it in the instance dictionary, so it will use that instead!

In [16]:
acc_1.apr, acc_2.apr

(0, 2.5)

In effect, the instance attribute `apr` is **hiding** the class attribute.

You'll notice also that `acc_2` was **not** affected - this is because we did not modify `acc_2`'s dictionary, just the dictionary for `acc_1`.

And the `getattr` and `setattr` functions work the same way as dotted notation:

In [17]:
acc_1 = BankAccount()
acc_2 = BankAccount()
acc_1.city = "Los Altos"
print(acc_1.__dict__)
print(acc_1.city)

{'city': 'Los Altos'}
Los Altos


In [18]:
print(getattr(acc_1, 'city'))
print(getattr(acc_2, 'city', "Attribute does not exist in object"))

Los Altos
Attribute does not exist in object


#### Where did `Attribute does not exist in object` come from?

In [19]:
help(getattr)

Help on built-in function getattr in module builtins:

getattr(...)
    getattr(object, name[, default]) -> value
    
    Get a named attribute from an object; getattr(x, 'y') is equivalent to x.y.
    When a default argument is given, it is returned when the attribute doesn't
    exist; without it, an exception is raised in that case.



#### Reruning the code cell from above

In [20]:
acc_1 = BankAccount()
acc_2 = BankAccount()
acc_1.city = "Los Altos"
print(acc_1.__dict__)
print(acc_1.city)
print(getattr(acc_1, 'city'))
print(getattr(acc_2, 'city', "Attribute does not exist in object"))

{'city': 'Los Altos'}
Los Altos
Los Altos
Attribute does not exist in object


In [21]:
setattr(acc_1, 'apr', 99)
print(f"acc_1 __dict__: {acc_1.__dict__}")
print(f"acc_1 apr: {acc_1.apr}") 
# Repeating the same statement
print(f"acc_1 apr: {getattr(acc_1, 'apr')}")

acc_1 __dict__: {'city': 'Los Altos', 'apr': 99}
acc_1 apr: 99
acc_1 apr: 99


We can even add instance attributes directly to an instance:

In [22]:
acc_1.bank = 'Acme Savings & Loans'

In [23]:
acc_1.__dict__

{'city': 'Los Altos', 'apr': 99, 'bank': 'Acme Savings & Loans'}

But this is specific to the instance, and only that specific instance:

In [24]:
acc_2 = BankAccount()

In [25]:
acc_2.__dict__

{}

As you can see `acc_2` has an empty instance dictionary.

So it is really important to distingush between **class attributes** and **instance attributes**.

**Class attributes** are like attributes that are "common" to all instances - because the attribute does not live in the instance, but in the class itself.

On the other hand, **instance attributes** are specific to each instance, and values for the same attribute can be different across multiple instances, as we just saw with `acc_1.apr` and `acc_2.apr`.

So, in summary,   
- classes and instances each have their own state - usually maintained in a dictionary, available through `__dict__`.    
- irrespective of where the state is stored, when we look up an attribute on an instance,     
 - Python will first look for the attribute in the instance's local state. 
 - If it does not find it there, it will next look for it in the class of the instance.

One other thing to note is the difference in type between class and instance `__dict__`.

Classes as we saw, return a `mapping proxy` object:

In [26]:
BankAccount.__dict__

mappingproxy({'__module__': '__main__',
              'apr': 2.5,
              '__dict__': <attribute '__dict__' of 'BankAccount' objects>,
              '__weakref__': <attribute '__weakref__' of 'BankAccount' objects>,
              '__doc__': None,
              'account_type': 'Savings'})

But instances, return a real dictionary:

In [27]:
acc_1.__dict__

{'city': 'Los Altos', 'apr': 99, 'bank': 'Acme Savings & Loans'}

So with instances, unlike with classes, we can manipulate that dictionary directly:

In [28]:
class Program:
    language = 'Python'

In [29]:
p = Program()

In [30]:
p.__dict__

{}

In [31]:
p.__dict__['version'] = '3.10'

In [32]:
p.__dict__

{'version': '3.10'}

In [33]:
p.version, getattr(p, 'version')

('3.10', '3.10')

But once again, this only affects that specific **instance**.